# AUTOML COM PYCARET - CLASSIFICAÇÃO EM FUNDOS IMOBILIÁRIOS

# Passo 1: Instalação Condicional de Bibliotecas e importação das bibliotecas 

In [23]:
## Criando e ativando um ambiente virtual
## O pycaret não funciona nas novas versões do python
## No terminal:
# conda create --prefix ./env python=3.10 -y
# conda activate ./env
# conda install -p "c:..." ipykernel --update-deps --force-reinstall


In [24]:
import subprocess
import os

# Instalar as bibliotecas do arquivo requirements.txt
# pip install -r requirements.txt

# Passo 2: Importação das Bibliotecas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from pycaret.classification import *

# Configuração global de plots
sns.set_style("whitegrid")

# Passo 3: Carregar os Dados

In [25]:
# Carregar os dados do Excel
df = pd.read_excel("seu_arquivo.xlsx")

# Exibir as primeiras linhas
df.head()

# Separar features e variável alvo
X = df.drop(columns=['target'])
y = df['target']

# Dividir em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Verificar as formas dos dados
print(f"Forma dos dados de treino: {X_train.shape}")
print(f"Forma dos dados de teste: {X_test.shape}")

FileNotFoundError: [Errno 2] No such file or directory: 'seu_arquivo.xlsx'

# Passo 4: Verificação de Consistência dos Dados

In [ ]:
# 4.1 Verificar informações gerais
X_train.info()

# 4.2 Estatísticas descritivas
X_train.describe()

# 4.3 Contagem de valores nulos
print("\nValores nulos por coluna:\n", X_train.isnull().sum())

# 4.4 Detectar variáveis constantes e quase constantes
def check_constant_features(X_train, threshold=0.99):
    """ Identifica variáveis com baixa variação """
    constant_features = []
    for col in X_train.select_dtypes(include=['number']).columns:
        top_freq = X_train[col].value_counts(normalize=True, dropna=False).values[0]
        if top_freq >= threshold:
            constant_features.append(col)
    return constant_features

const_features = check_constant_features(X_train)
print(f"\n🔴 Variáveis Quase Constantes (threshold=99%): {const_features}")

# 4.5 Verificar classes raras na variável alvo
if "target" in X_train.columns:
    class_counts = X_train["target"].value_counts(normalize=True)
    rare_classes = class_counts[class_counts < 0.05].index.tolist()
    print(f"\n🔴 Classes Raras (<5% de representatividade): {rare_classes}")

# 4.6 Matriz de Correlação
plt.figure(figsize=(12,6))
sns.heatmap(X_train.corr(), annot=True, cmap='coolwarm', fmt=".2f")
plt.title("Matriz de Correlação")
plt.show()


# Passo 5: Pré-Processamento e Feature Engineering

In [ ]:
# Lidar com valores ausentes (preenchimento com mediana)
X_train.fillna(X_train.median(), inplace=True)

# Remover variáveis constantes
X_train.drop(columns=const_features, inplace=True)

# Normalizar os dados
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Passo 6: Configuração do AutoML com PyCaret

In [ ]:
# Criar dataframe com treino para o PyCaret
X_train = pd.DataFrame(X_train_scaled, columns=X.columns)
X_train['target'] = y_train.values

# Inicializar a configuração do PyCaret
clf = setup(data=X_train, target='target', session_id=42, 
            normalize=True, 
            # Remover a linha categorical_features
            feature_selection=True, 
            remove_multicollinearity=True,
            transformation=True,
            fix_imbalance=True)

# Passo 7: Treinamento e Comparação de Modelos

In [ ]:
# Treinar e comparar diferentes modelos automaticamente
best_model = compare_models()

# Passo 8: Ajuste do Melhor Modelo

In [ ]:
# Criar o melhor modelo encontrado
final_model = create_model(best_model)

# Ajustar hiperparâmetros
tuned_model = tune_model(final_model)

# Passo 9: Avaliação do Modelo

In [ ]:
# Avaliar o modelo
evaluate_model(tuned_model)

# Exibir a importância das features
plot_model(tuned_model, plot='feature')

# Passo 10: Predições em Novos Dados

In [ ]:
# Fazer previsões nos dados de teste
df_test = pd.DataFrame(X_test_scaled, columns=X.columns)
df_test['target'] = y_test.values

predictions = predict_model(tuned_model, data=df_test)

# Exibir previsões
predictions.head()